# TP 3: Hasting-Metropolis (and Gibbs) samplers

In [1]:
# imports 
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(2024)

## Exercise 1: Hasting-Metropolis within Gibbs – Stochastic Approximation EM

#### 1. Write the complete log-likelihood of the previous model $ \log q(y, z, \theta ) $ and show that the proposed model belongs to the curved exponential family.

Cf pdf file.

#### 2. Generate synthetic data from the model by taking some reasonable values for the parameters.

In [2]:
def average_trajectory(p_0, v_0, t_0):
    """ Returns a function that gives the average trajectory
    with initial position p_0, initial velocity v_0, and initial time t_0."""
    return lambda t: p_0 + v_0 * (t - t_0)

def individual_trajectory(p_0, v_0, t_0, alpha_i, tau_i):
    """ Returns a function that gives the individual trajectory
    """
    average_trajectory = average_trajectory(p_0, v_0, t_0)
    return lambda t: average_trajectory(alpha_i * (t - t_0 - tau_i) + t_0)

sigma_p0 = 0.1
sigma_v0 = 0.1
sigma_t0 = 0.1

<function __main__.average_trajectory.<locals>.<lambda>(t)>